In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
import os
import json
import networkx as nx

In [45]:
# File Paths
absolute_path = "C:\\Users\\tykun\\\OneDrive\\Documents\\SchoolDocs\VSCodeProjects\\connectedData\\board_analysis\\"
altered_dataframes = "altered_dataframes\\"
gpt_dataframes = "gpt_dataframes\\"
graphs = "graphs\\"
scripts =  "scripts\\"
board_dataframes = "board_dataframes\\"
temporary_data = "temporary_data\\"
college_matching = "college_matching\\"

altered_dataframe_path = f"{absolute_path}{altered_dataframes}"
gpt_dataframe_path = f"{absolute_path}{gpt_dataframes}" 
graph_path = f"{absolute_path}{graphs}"
script_path = f"{absolute_path}{scripts}"
boards_path = f"{absolute_path}{board_dataframes}"
state_path = f"{absolute_path}{temporary_data}state_systems_validated.csv"   
college_matching_path = f"{absolute_path}{college_matching}"

# Valid Years
years = ["1999", "2000", "2005", "2007", "2008", "2009", "2011", "2013", "2018"]

In [46]:
def json_to_dataframes(json_file_path):
    # Load the JSON data
    with open(json_file_path, 'r') as file:
        data = json.load(file)
    
    # Extract nodes
    nodes = data.get("nodes", [])
    
    # Flatten nodes and attributes
    nodes_list = []
    for node in nodes:
        flattened_node = {
            "label": node.get("label"),
            "x": node.get("x"),
            "y": node.get("y"),
            "id": node.get("id"),
            "color": node.get("color"),
            "size": node.get("size"),
        }
        # Add all attributes from the "attributes" dictionary
        flattened_node.update(node.get("attributes", {}))
        nodes_list.append(flattened_node)
    
    # Convert to DataFrame
    nodes_df = pd.DataFrame(nodes_list)
    
    return nodes_df


In [47]:
board_statistics = pd.read_csv(f"{altered_dataframe_path}sample_board_statistics.csv")
network_statistics_path = f"{boards_path}interlock_stats.json"
network_statistics = json_to_dataframes(network_statistics_path)


In [48]:
def calculate_assortativity(nodes_df, edges_df, attribute):
    # Create a graph
    G = nx.Graph()
    
    # Add nodes with attributes
    for _, row in nodes_df.iterrows():
        G.add_node(row['Id'], **row.to_dict())
    
    # Add edges
    for _, row in edges_df.iterrows():
        G.add_edge(row['Source'], row['Target'])
    
    # Calculate assortativity for the given attribute
    assortativity = nx.attribute_assortativity_coefficient(G, attribute)
    return assortativity


nodes_df = pd.read_csv(f"{absolute_path}{board_dataframes}aggregated_nodes.csv")
edges_df = pd.read_csv(f"{absolute_path}{board_dataframes}aggregated_edges.csv")
assortativity = calculate_assortativity(nodes_df, edges_df, 'region')

print(f"Assortativity based on 'female_president': {assortativity}")

Assortativity based on 'female_president': 0.3839321203202145
